#### Defined by User

In [ ]:
## specify your Dkube username
DKUBEUSERNAME = "ocdkube"

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "precomputed-mm"
## Specify the data source [aws-s3,sql,local]
PRECOMPUTED_DATA_SOURCE = "local"

PRESCORE_DATASET = f"{MONITOR_NAME}-{PRECOMPUTED_DATA_SOURCE}"
## Dkube information
import os
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")

## AWS_S3 
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID","")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY","")
BUCKET = os.getenv("BUCKET","")
## SQL
# DBHOSTNAME should be provided in the one of the following format.
# IP:PORT for eg 127.0.0.1:3306
# DOMAIN:PORT for eg my-db.com:3306
DBHOSTNAME   = os.getenv("DBHOSTNAME","")
DATABASENAME = os.getenv("DATABASENAME","")
DBUSERNAME   = os.getenv("DBUSERNAME","")
DBPASSWORD   = os.getenv("DBPASSWORD","")


# supported value are mysql or mssql
DB_PROVIDER = "mysql"

# the frequency with which monitoring will run
RUN_FREQUENCY = 5

if TOKEN == '' or DKUBEUSERNAME == '' or DKUBE_URL == '':
    print("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME)")
    raise TypeError
if PRECOMPUTED_DATA_SOURCE == 'aws_s3' and (ACCESS_KEY == '' or SECRET_KEY == '' or BUCKET == ''):
    print("Please fill the AWS_S3 details first (ACCESS_KEY, SECRET_KEY, BUCKET)")
    raise TypeError
if PRECOMPUTED_DATA_SOURCE == 'sql' and (DBHOSTNAME == '' or DATABASENAME == '' or DBUSERNAME == '' or DBPASSWORD == ''):
    print("Please fill the SQL details first (DBHOSTNAME, DATABASENAME, DBUSERNAME, DBPASSWORD)")
    raise TypeError
if PRECOMPUTED_DATA_SOURCE == "sql" and DB_PROVIDER not in ["mysql", "mssql"]:
    raise ValueError(f"{DB_PROVIDER} is not supported")

In [ ]:
import time,json,shutil
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
data = api.get_modelmonitor_id(MONITOR_NAME)
if data.data:
    MONITOR_ID = data.data.get(MONITOR_NAME)
    if MONITOR_ID:
        raise ValueError(f"{MONITOR_NAME} already existing please use a different name")

In [ ]:
ps_config = {"MONITOR_NAME":MONITOR_NAME, "PRECOMPUTED_DATA_SOURCE": PRECOMPUTED_DATA_SOURCE, 
            "DKUBEUSERNAME":DKUBEUSERNAME, "TOKEN":TOKEN, "DKUBE_URL":DKUBE_URL, 
            "ACCESS_KEY":ACCESS_KEY, "SECRET_KEY":SECRET_KEY, "BUCKET":BUCKET,
            "DBHOSTNAME":DBHOSTNAME, "DATABASENAME":DATABASENAME,
            "DBUSERNAME":DBUSERNAME, "DBPASSWORD":DBPASSWORD, "DB_PROVIDER":DB_PROVIDER,
            "PRESCORE_DATASET":PRESCORE_DATASET, "RUN_FREQUENCY":RUN_FREQUENCY,
            }
%store ps_config

In [ ]:
def get_dataset_version(username, dataset_name, version):
    dataset_versions = api.get_dataset_versions(username, dataset_name)
    versions = []
    for each_version in dataset_versions:
        if each_version["version"]["name"] == version:
            uuid = each_version["version"]["uuid"]
            return f"{version}:{uuid}"
        else:
            versions.append(each_version["version"]["name"])
    return f"dataset version {version} not found, available version are {versions}"

# Adding  precomputed labelled data source

In [ ]:
if PRECOMPUTED_DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=PRESCORE_DATASET)
        api.create_dataset(dataset)
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
        
elif PRECOMPUTED_DATA_SOURCE == 'sql':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=PRESCORE_DATASET,remote=True)
        dataset.update_dataset_source('sql')
        dataset.update_sql_details(
            host=DBHOSTNAME.split(":")[0],
            port=int(DBHOSTNAME.split(":")[1]),
            username=DBUSERNAME,
            password=DBPASSWORD,
            database=DATABASENAME,
            provider=DB_PROVIDER)
        api.create_dataset(dataset)
     
    except Exception as e:
        if e.reason:
            if e.reason.lower() != "conflict":
                response = e.body
                print(f"Failed[{response.code}]: {response.message}")
        else:
            raise e
elif PRECOMPUTED_DATA_SOURCE == 'aws-s3':
    try:
        dataset = DkubeDataset(DKUBEUSERNAME, name=PRESCORE_DATASET,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(
            bucket=BUCKET,
            prefix="",key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),
            secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason:
            if e.reason.lower() != "conflict":
                response = e.body
                print(f"Failed[{response.code}]: {response.message}")
        else:
            raise e

# Creating Model Monitor

In [ ]:
import requests, operator
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo, DataType
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitorAlert, TimeZone
from dkube.sdk.rsrcs.modelmonitor import SourceTypePerformance

### Importing deployment

In [ ]:
try:
    api.import_deployment(name=MONITOR_NAME)
except BaseException as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e
DEPLOYMENT_ID = api.get_deployment_id(name=MONITOR_NAME)

In [ ]:
with open('thresholds.json') as f:
    thresholds = json.load(f)

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Classification.value, 
                               input_data_type=DataType.Tabular.value,
                               data_timezone=TimeZone.UTC.value)

mm.add_thresholds(thresholds=thresholds)

In [ ]:
labelled_data = f"{PRESCORE_DATASET}:{DKUBEUSERNAME}"
labelled_data_version = get_dataset_version(DKUBEUSERNAME,PRESCORE_DATASET, "v1")

if PRECOMPUTED_DATA_SOURCE == 'sql':
    mm.add_datasources(data_class=DatasetClass.Metrics.value,
                       name=labelled_data,data_format=str(DatasetFormat.Tabular),
                       sql_query="select * from insurance_precomputed")

if PRECOMPUTED_DATA_SOURCE == 'local':
    mm.add_datasources(data_class=DatasetClass.Metrics.value,
                       name=labelled_data,data_format=str(DatasetFormat.Tabular),
                       version=labelled_data_version)

if PRECOMPUTED_DATA_SOURCE == 'aws-s3':
    mm.add_datasources(data_class=DatasetClass.Metrics.value,
                       name=labelled_data, s3_subpath=MONITOR_NAME + "/pre_computed_scores",
                       data_format=str(DatasetFormat.Tabular))

In [ ]:
mm.update_performance_monitoring_details(enabled=True,
                                         source_type=SourceTypePerformance.Metrics.value,
                                         frequency=RUN_FREQUENCY)

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

## Alerts

In [ ]:
alert = DkubeModelmonitorAlert(name='perf_alert', alert_class = 'performance_decay')
alert.add_alert_condition(metric='accuracy',threshold=0.9, op=operator.lt)
api.modelmonitor_add_alert(DEPLOYMENT_ID,alert)

In [ ]:
api.modelmonitor_start(DEPLOYMENT_ID)

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_dataset(DKUBEUSERNAME,PRESCORE_DATASET,force=True)
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(DEPLOYMENT_ID)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(DEPLOYMENT_ID)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(DEPLOYMENT_ID)
    %store -d ps_config